# Plaintext classification neural network model

In the following code, the plaintext neural network model is implemented. Before the useage of the model, the data is cleaned based on the code from the ADML module at Hochschule Luzern by Solange Emmenegger (Solange Emmenegger, Hochschule Luzern, Module Advanced Machine Learning, accessed on 19 April 2024 at https://gitlab.renku.hslu.ch/solange.emmenegger/ml-adml-hslu/-/tree/master/notebooks/03A%20Supervised%20Learning, and https://gitlab.renku.hslu.ch/solange.emmenegger/ml-adml-hslu/-/blob/master/notebooks/04B%20Gradient%20Descent/Gradient%20Descent.ipynb) and modified where necessary. 

The model architecture was build by the authour with assistance from the Claude, ChatGPT and Gemini Chatbots. (“Please suggest a PyTorch neural network architecture for binary classification”, Claude 3, Anthropic PBC, generated on 27 March 2024., “Please improve this neural network architecture for binary classification…”, ChatGPT (GPT-4), OpenAI, generated on 27 March 2024., “Why is the F2 score of the crypten model worse and how do I improve it”, Gemini, Google (Alphabet Inc.), generated on 30 March 2024. )

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import fbeta_score

from time import time
import psutil

import torch
from torch import nn
from torch.optim import Adam
from tqdm.notebook import tqdm
import os


os.environ["CUDA_VISIBLE_DEVICES"]=""

## Cleaning and preparation of the dataset

In the code below, the dataset is split into a training and test set, it is scaled and trimmed to 10000 samples, which is the size used for the evaluation. This size was chosen to avoid the model training for days, as the author of the project has constrained hardware resources.

In [2]:
df_nn = pd.read_csv("card_transdata.csv")

train_transactions, test_transactions = train_test_split(df_nn, test_size=0.2, random_state=42)
print(train_transactions[:10000][train_transactions['fraud'] == 1.0].shape)

X_train_transactions = train_transactions.drop(columns=["fraud"])
y_train_transactions = train_transactions.fraud.values
X_test_transactions = test_transactions.drop(columns=["fraud"])
y_test_transactions = test_transactions.fraud.values

scaler = StandardScaler()
X_train_transactions = pd.DataFrame(scaler.fit_transform(X_train_transactions), columns=X_train_transactions.columns, index=X_train_transactions.index).values
X_test_transactions = pd.DataFrame(scaler.transform(X_test_transactions), columns=X_test_transactions.columns, index=X_test_transactions.index).values

print(f"X_train_transactions has shape: {X_train_transactions.shape}")
print(f"y_train_transactions has shape: {y_train_transactions.shape}")
print(f"X_test_transactions has shape: {X_test_transactions.shape}")
print(f"y_test_transactions has shape: {y_test_transactions.shape}")

(855, 8)
X_train_transactions has shape: (800000, 7)
y_train_transactions has shape: (800000,)
X_test_transactions has shape: (200000, 7)
y_test_transactions has shape: (200000,)


/tmp/ipykernel_87107/423273962.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(train_transactions[:10000][train_transactions['fraud'] == 1.0].shape)


## Neural network architecture

Below is the neural network architecture, as described in the report. 

In [3]:
class FraudDetectionModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes, lr=0.001):
        super(FraudDetectionModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size, dtype=torch.float64)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)  # Dropout layer to prevent overfitting
        self.bn1 = nn.BatchNorm1d(hidden_size, dtype=torch.float64)
        self.fc2 = nn.Linear(hidden_size, hidden_size, dtype=torch.float64)
        self.fc3 = nn.Linear(hidden_size, num_classes, dtype=torch.float64)
        self.bn2 = nn.BatchNorm1d(num_classes, dtype=torch.float64)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.bn1(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.bn1(x)
        x = self.fc3(x)
        x = self.bn2(x)
        x = self.sigmoid(x)
        return x


## Define the model hyperparameters, cost function and the learning rate

Below we select the device used for model training. If a NVIDIA GPU is available, the GPU is used, otherwise the CPU is used.
We have the input size of 7 as we have seven features. The hidden size of 64 is chosen, which is a hyperparameter and is chosen heuristically by experimentation. 
Number of classes is set to 1, which indicates binary classification. 

We train the model over 1000 epochs, which was chosen for the evaluation.

In [4]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# Initialize model parameters
input_size = 7  # Number of input features
hidden_size = 64  # Number of hidden units
num_classes = 1  # Binary classification
num_epochs = 5

model = FraudDetectionModel(input_size, hidden_size, num_classes).to(device)

criterion = nn.BCELoss()  # Binary Cross-Entropy Loss for binary classification
optimizer = Adam(model.parameters(), lr=0.001)

Using cpu device


## Training function and training

Below the training function and its execution is defined, which is the usual format for training a pytorch neural network model.

In [5]:
def train(model, optim, criterion, x, y, epochs=num_epochs):
    batch_size = 32 
    num_batches = x.size(0) // batch_size

    for e in tqdm(range(1, epochs + 1)):
            
        for batch in range(num_batches):
            # define the start and end of the training mini-batch
            start, end = batch * batch_size, (batch + 1) * batch_size
            x_train = x[start:end]
            y_train = y[start:end]
            optim.zero_grad()
            out = model(x_train.to(device))
            loss = criterion(out, y_train.to(device))
            loss.backward()
            optim.step()
        accuracy = accuracy_score(y_train.detach().numpy(),  np.where(out.detach().numpy() > 0.5, 1, 0))
        f2_score = fbeta_score(y_train.detach().numpy(), np.where(out.detach().numpy() > 0.5, 1, 0), beta=0.5)
        print(f"Loss at epoch {e}: {loss.data}")
        print(f"Accuracy at epoch {e}: {accuracy}")
        print(f"F2 score at epoch {e}: {f2_score}")
    return model



In [6]:
len_size = 50000

# measure time
t_start = time()
# measure resource usage
mem_usage = psutil.Process().memory_info().rss
model = train(model, optimizer, criterion, torch.from_numpy(X_train_transactions[:len_size]), torch.from_numpy(y_train_transactions[:len_size]).view(-1, 1))
mem_usage_end = psutil.Process().memory_info().rss

# Calculate the differences
mem_diff = mem_usage_end - mem_usage
t_end = time()
print(f"Training of the neural network took {int(t_end - t_start)} seconds")
print(f"Memory usage difference: {mem_diff} bytes")

  0%|          | 0/5 [00:00<?, ?it/s]

Loss at epoch 1: 0.21605794707204967
Accuracy at epoch 1: 1.0
F2 score at epoch 1: 1.0
Loss at epoch 2: 0.12521208433681993
Accuracy at epoch 2: 0.96875
F2 score at epoch 2: 0.7142857142857143
Loss at epoch 3: 0.04226166487500828
Accuracy at epoch 3: 1.0
F2 score at epoch 3: 1.0
Loss at epoch 4: 0.03752758370388637
Accuracy at epoch 4: 1.0
F2 score at epoch 4: 1.0
Loss at epoch 5: 0.09614952146572958
Accuracy at epoch 5: 0.96875
F2 score at epoch 5: 0.8333333333333334
Training of the neural network took 7 seconds
Memory usage difference: 11341824 bytes


Finally, the model is evaluated on the test set.


In [7]:
y_pred = model(torch.from_numpy(X_test_transactions).to(device))
y_pred = np.where(y_pred.detach().numpy() > 0.5, 1, 0)
print("Accuracy score: {}".format(accuracy_score(y_test_transactions, y_pred)))
print("F2 score: {}".format(fbeta_score(y_test_transactions, y_pred=y_pred, beta=0.5)))

Accuracy score: 0.9856
F2 score: 0.9479582146248813


Below, the helper function for visualisation of the accuracy and F2 scores is defined and the scores are visualised.

In [8]:
def plot_validation_curve(data, ax=None, ylim=None):
    if ax is None:
        fig, ax = plt.subplots()
        ax.set_title("Validation Curve")
        ax.set_ylabel("Cost")
    if ylim is not None:
        ax.set_ylim(ylim)
    ax.set_xlabel("Epochs")
    ax.plot(data)


def plot_validation_curves_acc(hist, ylim=None):
    fig, ax = plt.subplots(ncols=2, figsize=(16,5))

    ax[0].set_title("Train Cost")
    ax[0].set_ylabel("Cost")
    plot_validation_curve(hist["train_cost"], ax[0], ylim)

    ax[1].set_title("Train accuracy")
    ax[1].set_ylabel("F2")
    ax[1].set_ylim(-1, 1)
    plot_validation_curve(hist["train_acc"], ax[1])

    plt.tight_layout()

plot_validation_curves_acc(hist)

NameError: name 'hist' is not defined

In [ ]:
def plot_validation_curves_f2(hist, ylim=None):
    fig, ax = plt.subplots(ncols=2, figsize=(16,5))

    ax[0].set_title("Train Cost")
    ax[0].set_ylabel("Cost")
    plot_validation_curve(hist["train_cost"], ax[0], ylim)

    ax[1].set_title("Train F2")
    ax[1].set_ylabel("F2")
    ax[1].set_ylim(-1, 1)
    plot_validation_curve(hist["train_f2"], ax[1])

    plt.tight_layout()

plot_validation_curves_f2(hist)